In [1]:
import numpy as np

import tensorflow as tf

tfd = tf.contrib.distributions


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Importing data

In [0]:
data_path = "gene_expression.txt"

expression=np.loadtxt(data_path )# observed data
#labels
label = np.loadtxt("labels.txt")

In [8]:


n_y=558   #dim of observed data
n_z=10   #dim of z
learn_rate=0.0001
logW= tf.Variable(np.random.normal(size=n_y))# log of W
loglambda=tf.Variable(np.random.normal(size=1))#log of lambda
print(logW.shape)

(558,)


In [0]:
def get_batches(X, batch_size = 100): # get batches of traing data
    
	
    n_batches =len(X)//batch_size
    X = X[:n_batches*batch_size]

	# Loop over batches and yield
    for b in range(0, len(X), batch_size):
        yield X[b:b+batch_size]

In [0]:
def encoder(x):
    mu = tf.layers.dense(x, n_z)
    logsigma = tf.layers.dense(x, n_z)
    return mu, logsigma


#decoder
def decoder(z):
    f_z=tf.layers.dense(z, n_y)
    return f_z

In [11]:
data= tf.placeholder(tf.float32, [None, n_y], name = 'y')

#z posterior sampling
def sampling_z( mu, logsigma):
    std = tf.exp(0.5*logsigma)
    eps = tf.random_normal(shape=[tf.shape(data)[0],
                              n_z],
                       mean=0.0,
                       stddev=1.0)
    return mu + tf.sqrt(std) * eps
# posterior of z given y
z_mu, z_logsigma = encoder(data)
print(z_logsigma.shape)
z = sampling_z(z_mu, z_logsigma)
print(z)
x_tilde = decoder(z)
W = tf.exp(logW)
lam_bda=tf.exp(loglambda)

(?, 10)
Tensor("add_1:0", shape=(?, 10), dtype=float32)


In [12]:
W=tf.cast(W,tf.float32)
lam_bda=tf.cast(lam_bda,tf.float32)
er=1e-8
l_zero =  -tf.log(1+2*W*lam_bda)/2 - x_tilde*x_tilde*lam_bda/(1+2*W*lam_bda)
print(l_zero.shape)
l_nonzero=tf.log(1-tf.exp(-lam_bda*data*data)+er)  - tf.log(W+er)/2 - (data-x_tilde)*(data-x_tilde)/(2*W)
l=tf.multiply(tf.cast(data<er,tf.float32),l_zero)+tf.multiply(tf.cast(data>=er,tf.float32),l_nonzero)
Y = tf.distributions.Normal(tf.zeros_like(z_mu), tf.ones_like(z_logsigma))
X = tf.distributions.Normal(z_mu, tf.sqrt(tf.exp(z_logsigma)))
     
kl_z =tf.distributions.kl_divergence(X, Y)
print(kl_z)     
lossv = - tf.reduce_sum(l,1) +tf.reduce_sum(kl_z,1) 
loss=(tf.reduce_mean(lossv))
train_op = tf.train.AdamOptimizer(learn_rate).minimize(loss)

(?, 558)
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Tensor("KullbackLeibler/kl_normal_normal/add:0", shape=(?, 10), dtype=float32)
Instructions for updating:
Use tf.cast instead.


In [15]:

sess=tf.Session()
iteration=1
sess.run(tf.global_variables_initializer())
los_old=0
for i in range(800):
   for X in get_batches(expression, 100):#batch_size=100
           
                sess.run(train_op,{data: X})
                iteration=iteration+1
    
            
            # Print at each 5 iters
                if (iteration % 10 == 0):
                  los=sess.run(loss,{data: expression})
                  print(los)
                
                  if abs(los-los_old)<0.05:
                    break
                  los_old=los
  

1873.9148
1829.9891
1804.5704
1783.8049
1756.0785
1734.9641
1710.2847
1672.8864
1643.9988
1611.3813
1564.2672
1528.9985
1490.8442
1438.1738
1399.8845
1360.547
1307.5684
1270.621
1233.5317
1185.9211
1151.8768
1118.7302
1078.5399
1050.1117
1023.87463
989.1563
966.8219
944.65717
916.748
898.7864
881.3999
859.4592
844.13776
830.0749
812.8591
801.0859
789.2168
775.3144
766.21466
756.7644
745.7424
737.60333
729.51184
720.5184
713.53687
707.2722
699.88794
694.0796
688.99457
681.8223
677.55597
672.66156
667.3692
662.74
658.63745
654.4652
650.67444
647.2608
642.8144
639.31665
637.223
632.821
630.5521
627.41815
623.68396
621.71063
618.5184
615.8752
613.80145
611.05963
608.5518
606.0004
604.61163
602.00684
600.36163
598.31323
595.75287
594.1795
592.055
590.30304
588.2694
586.8684
585.167
583.455
581.32434
579.94666
578.3721
576.666
575.34515
573.73
572.1725
570.86176
569.4272
568.0292
566.7796
565.0634
563.63434
562.8019
561.4377
560.1983
558.94556
557.87756
555.9764
555.14124
554.03656
552.7773


In [0]:
Z=sess.run(z,{data: expression})

In [17]:

from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.metrics import adjusted_rand_score as ARI

labels_pred_Z = KMeans(7, n_jobs=8, n_init=100).fit_predict(Z)
print("Tthe NMI and ARI is")
[NMI(label, labels_pred_Z), ARI(label, labels_pred_Z)]

Tthe NMI and ARI is


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


[0.7472862066111651, 0.7066735840334922]